In [6]:
import io
#使用 io.StringIO() 和 io.BytesIO() 类来创建类文件对象操作字符串数据
s = io.StringIO()
s.write('Hello World\n')
#io.StringIO 只能用于文本。如果你要操作二进制数据，要使用 io.BytesIO 类来代替

12

你想直接读取二进制数据到一个可变缓冲区中，而不需要做任何的中间复制操作。
或者你想原地修改数据并将它写回到一个文件中去。

### 解决方案


为了读取数据到一个可变数组中，使用文件对象的 readinto() 方法。比如：

In [ ]:
import os.path

def read_into_buffer(filename):
    buf = bytearray(os.path.getsize(filename))
    with open(filename, 'rb') as f:
        f.readinto(buf)
    return buf

下面是一个演示这个函数使用方法的例子：

In [ ]:
# Write a sample file
with open('sample.bin', 'wb') as f:
    f.write(b'Hello World')
buf = read_into_buffer('sample.bin')
buf

### 讨论


文件对象的 readinto() 方法能被用来为预先分配内存的数组填充数据，甚至包括由 array 模块或 numpy 库创建的数组。
和普通 read() 方法不同的是， readinto() 填充已存在的缓冲区而不是为新对象重新分配内存再返回它们。
因此，你可以使用它来避免大量的内存分配操作。
比如，如果你读取一个由相同大小的记录组成的二进制文件时，你可以像下面这样写：

In [ ]:
record_size = 32 # Size of each record (adjust value)

buf = bytearray(record_size)
with open('somefile', 'rb') as f:
    while True:
        n = f.readinto(buf)
        if n < record_size:
            break
        # Use the contents of buf
        ...

另外有一个有趣特性就是 memoryview ，
它可以通过零复制的方式对已存在的缓冲区执行切片操作，甚至还能修改它的内容。比如：



使用 f.readinto() 时需要注意的是，你必须检查它的返回值，也就是实际读取的字节数。

如果字节数小于缓冲区大小，表明数据被截断或者被破坏了(比如你期望每次读取指定数量的字节)。



## 5.10 内存映射的二进制文件


### 问题


你想内存映射一个二进制文件到一个可变字节数组中，目的可能是为了随机访问它的内容或者是原地做些修改。

### 解决方案


使用 mmap 模块来内存映射文件。
下面是一个工具函数，向你演示了如何打开一个文件并以一种便捷方式内存映射这个文件。

In [ ]:
import os
import mmap

def memory_map(filename, access=mmap.ACCESS_WRITE):
    size = os.path.getsize(filename)
    fd = os.open(filename, os.O_RDWR)
    return mmap.mmap(fd, size, access=access)

为了使用这个函数，你需要有一个已创建并且内容不为空的文件。
下面是一个例子，教你怎样初始创建一个文件并将其内容扩充到指定大小：

In [ ]:
size = 1000000
with open('data', 'wb') as f:
    f.seek(size-1)
    f.write(b'\x00')

下面是一个利用 memory_map() 函数类内存映射文件内容的例子：

In [ ]:
m = memory_map('data')
len(m)

In [ ]:
m[0:10]

In [ ]:
m[0]

In [ ]:
# Reassign a slice
m[0:11] = b'Hello World'
m.close()

In [ ]:
# Verify that changes were made
with open('data', 'rb') as f:
print(f.read(11))

mmap() 返回的 mmap 对象同样也可以作为一个上下文管理器来使用，
这时候底层的文件会被自动关闭。比如：

In [ ]:
with memory_map('data') as m:
    print(len(m))
    print(m[0:10])

In [ ]:
m.closed

默认情况下， memeory_map() 函数打开的文件同时支持读和写操作。
任何的修改内容都会复制回原来的文件中。
如果需要只读的访问模式，可以给参数 access 赋值为 mmap.ACCESS_READ 。比如：

In [ ]:
m = memory_map(filename, mmap.ACCESS_READ)

如果你想在本地修改数据，但是又不想将修改写回到原始文件中，可以使用 mmap.ACCESS_COPY ：

In [ ]:
m = memory_map(filename, mmap.ACCESS_COPY)

### 讨论


为了随机访问文件的内容，使用 mmap 将文件映射到内存中是一个高效和优雅的方法。
例如，你无需打开一个文件并执行大量的 seek() ， read() ， write() 调用，
只需要简单的映射文件并使用切片操作访问数据即可。

一般来讲， mmap() 所暴露的内存看上去就是一个二进制数组对象。
但是，你可以使用一个内存视图来解析其中的数据。比如：

In [ ]:
m = memory_map('data')
# Memoryview of unsigned integers
v = memoryview(m).cast('I')
v[0] = 7
m[0:4]

In [ ]:
m[0:4] = b'\x07\x01\x00\x00'
v[0]

需要强调的一点是，内存映射一个文件并不会导致整个文件被读取到内存中。
也就是说，文件并没有被复制到内存缓存或数组中。相反，操作系统仅仅为文件内容保留了一段虚拟内存。
当你访问文件的不同区域时，这些区域的内容才根据需要被读取并映射到内存区域中。
而那些从没被访问到的部分还是留在磁盘上。所有这些过程是透明的，在幕后完成！

如果多个Python解释器内存映射同一个文件，得到的 mmap 对象能够被用来在解释器直接交换数据。
也就是说，所有解释器都能同时读写数据，并且其中一个解释器所做的修改会自动呈现在其他解释器中。
很明显，这里需要考虑同步的问题。但是这种方法有时候可以用来在管道或套接字间传递数据。

这一小节中函数尽量写得很通用，同时适用于Unix和Windows平台。
要注意的是使用 mmap() 函数时会在底层有一些平台的差异性。
另外，还有一些选项可以用来创建匿名的内存映射区域。
如果你对这个感兴趣，确保你仔细研读了Python文档中
这方面的内容 。

## 5.11 文件路径名的操作


### 问题


你需要使用路径名来获取文件名，目录名，绝对路径等等。

### 解决方案


使用 os.path 模块中的函数来操作路径名。
下面是一个交互式例子来演示一些关键的特性：

In [ ]:
import os
path = '/Users/beazley/Data/data.csv'

In [ ]:
# Get the last component of the path
os.path.basename(path)

In [ ]:
# Get the directory name
os.path.dirname(path)

In [ ]:
# Join path components together
os.path.join('tmp', 'data', os.path.basename(path))

In [ ]:
# Expand the user's home directory
path = '~/Data/data.csv'
os.path.expanduser(path)

In [ ]:
# Split the file extension
os.path.splitext(path)

In [ ]:
#对于文件名的匹配，你可能会考虑使用 glob 或 fnmatch 模块。
import glob
pyfiles = glob.glob('somedir/*.py')

from fnmatch import fnmatch
pyfiles = [name for name in os.listdir('somedir')
            if fnmatch(name, '*.py')]

默认情况下，所有的文件名都会根据 sys.getfilesystemencoding() 返回的文本编码来编码或解码。比如：

In [1]:
import sys
sys.getfilesystemencoding()

'utf-8'

In [ ]:
#当打印未知的文件名时，使用下面的方法可以避免这样的错误
def bad_filename(filename):
    return repr(filename)[1:-1]

try:
    print(filename)
except UnicodeEncodeError:
    print(bad_filename(filename))

### 讨论


这一小节讨论的是在编写必须处理文件系统的程序时一个不太常见但又很棘手的问题。
默认情况下，Python假定所有文件名都已经根据 sys.getfilesystemencoding() 的值编码过了。
但是，有一些文件系统并没有强制要求这样做，因此允许创建文件名没有正确编码的文件。
这种情况不太常见，但是总会有些用户冒险这样做或者是无意之中这样做了(
可能是在一个有缺陷的代码中给 open() 函数传递了一个不合规范的文件名)。

当执行类似 os.listdir() 这样的函数时，这些不合规范的文件名就会让Python陷入困境。
一方面，它不能仅仅只是丢弃这些不合格的名字。而另一方面，它又不能将这些文件名转换为正确的文本字符串。
Python对这个问题的解决方案是从文件名中获取未解码的字节值比如 \xhh
并将它映射成Unicode字符 \udchh 表示的所谓的”代理编码”。
下面一个例子演示了当一个不合格目录列表中含有一个文件名为bäd.txt(使用Latin-1而不是UTF-8编码)时的样子：

## 5.16 增加或改变已打开文件的编码


### 问题


你想在不关闭一个已打开的文件前提下增加或改变它的Unicode编码。

### 解决方案


如果你想给一个以二进制模式打开的文件添加Unicode编码/解码方式，
可以使用 io.TextIOWrapper() 对象包装它。比如：

In [ ]:
import urllib.request
import io

u = urllib.request.urlopen('http://www.python.org')
f = io.TextIOWrapper(u, encoding='utf-8')
text = f.read()

如果你想修改一个已经打开的文本模式的文件的编码方式，可以先使用 detach() 方法移除掉已存在的文本编码层，
并使用新的编码方式代替。下面是一个在 sys.stdout 上修改编码方式的例子：

In [ ]:
import sys
sys.stdout.encoding

In [ ]:
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')
sys.stdout.encoding

这样做可能会中断你的终端，这里仅仅是为了演示而已。

### 讨论


I/O系统由一系列的层次构建而成。你可以试着运行下面这个操作一个文本文件的例子来查看这种层次：

In [ ]:
f = open('sample.txt','w')
f

In [ ]:
f.buffer

In [ ]:
f.buffer.raw

在这个例子中，io.TextIOWrapper 是一个编码和解码Unicode的文本处理层，
io.BufferedWriter 是一个处理二进制数据的带缓冲的I/O层，
io.FileIO 是一个表示操作系统底层文件描述符的原始文件。
增加或改变文本编码会涉及增加或改变最上面的 io.TextIOWrapper 层。

一般来讲，像上面例子这样通过访问属性值来直接操作不同的层是很不安全的。
例如，如果你试着使用下面这样的技术改变编码看看会发生什么：

In [ ]:
f

In [ ]:
f = io.TextIOWrapper(f.buffer, encoding='latin-1')
f

In [ ]:
f.write('Hello')

结果出错了，因为f的原始值已经被破坏了并关闭了底层的文件。

detach() 方法会断开文件的最顶层并返回第二层，之后最顶层就没什么用了。例如：

In [ ]:
f = open('sample.txt', 'w')
f

In [ ]:
b = f.detach()
b

In [ ]:
f.write('hello')

一旦断开最顶层后，你就可以给返回结果添加一个新的最顶层。比如：

In [ ]:
f = io.TextIOWrapper(b, encoding='latin-1')
f

尽管已经向你演示了改变编码的方法，
但是你还可以利用这种技术来改变文件行处理、错误机制以及文件处理的其他方面。例如：

In [ ]:
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='ascii',
                            errors='xmlcharrefreplace')
print('Jalape\u00f1o')

注意下最后输出中的非ASCII字符 ñ 是如何被 &#241; 取代的。

## 5.17 将字节写入文本文件


### 问题


你想在文本模式打开的文件中写入原始的字节数据。

### 解决方案


将字节数据直接写入文件的缓冲区即可，例如：

In [ ]:
import sys
sys.stdout.write(b'Hello\n')

In [ ]:
sys.stdout.buffer.write(b'Hello\n')

类似的，能够通过读取文本文件的 buffer 属性来读取二进制数据。

### 讨论


I/O系统以层级结构的形式构建而成。
文本文件是通过在一个拥有缓冲的二进制模式文件上增加一个Unicode编码/解码层来创建。
buffer 属性指向对应的底层文件。如果你直接访问它的话就会绕过文本编码/解码层。

本小节例子展示的 sys.stdout 可能看起来有点特殊。
默认情况下，sys.stdout 总是以文本模式打开的。
但是如果你在写一个需要打印二进制数据到标准输出的脚本的话，你可以使用上面演示的技术来绕过文本编码层。

## 5.18 将文件描述符包装成文件对象


### 问题


你有一个对应于操作系统上一个已打开的I/O通道(比如文件、管道、套接字等)的整型文件描述符，
你想将它包装成一个更高层的Python文件对象。

### 解决方案


一个文件描述符和一个打开的普通文件是不一样的。
文件描述符仅仅是一个由操作系统指定的整数，用来指代某个系统的I/O通道。
如果你碰巧有这么一个文件描述符，你可以通过使用 open() 函数来将其包装为一个Python的文件对象。
你仅仅只需要使用这个整数值的文件描述符作为第一个参数来代替文件名即可。例如：

In [ ]:
# Open a low-level file descriptor
import os
fd = os.open('somefile.txt', os.O_WRONLY | os.O_CREAT)

# Turn into a proper file
f = open(fd, 'wt')
f.write('hello world\n')
f.close()

当高层的文件对象被关闭或者破坏的时候，底层的文件描述符也会被关闭。
如果这个并不是你想要的结果，你可以给 open() 函数传递一个可选的 colsefd=False 。比如：

In [ ]:
# Create a file object, but don't close underlying fd when done
f = open(fd, 'wt', closefd=False)
...

### 讨论


在Unix系统中，这种包装文件描述符的技术可以很方便的将一个类文件接口作用于一个以不同方式打开的I/O通道上，
如管道、套接字等。举例来讲，下面是一个操作管道的例子：

In [ ]:
from socket import socket, AF_INET, SOCK_STREAM

def echo_client(client_sock, addr):
    print('Got connection from', addr)

    # Make text-mode file wrappers for socket reading/writing
    client_in = open(client_sock.fileno(), 'rt', encoding='latin-1',
                closefd=False)

    client_out = open(client_sock.fileno(), 'wt', encoding='latin-1',
                closefd=False)

    # Echo lines back to the client using file I/O
    for line in client_in:
        client_out.write(line)
        client_out.flush()

    client_sock.close()

def echo_server(address):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(1)
    while True:
        client, addr = sock.accept()
        echo_client(client, addr)

需要重点强调的一点是，上面的例子仅仅是为了演示内置的 open() 函数的一个特性，并且也只适用于基于Unix的系统。
如果你想将一个类文件接口作用在一个套接字并希望你的代码可以跨平台，请使用套接字对象的 makefile() 方法。
但是如果不考虑可移植性的话，那上面的解决方案会比使用 makefile() 性能更好一点。

你也可以使用这种技术来构造一个别名，允许以不同于第一次打开文件的方式使用它。
例如，下面演示如何创建一个文件对象，它允许你输出二进制数据到标准输出(通常以文本模式打开)：

In [ ]:
import sys
# Create a binary-mode file for stdout
bstdout = open(sys.stdout.fileno(), 'wb', closefd=False)
bstdout.write(b'Hello World\n')
bstdout.flush()

尽管可以将一个已存在的文件描述符包装成一个正常的文件对象，
但是要注意的是并不是所有的文件模式都被支持，并且某些类型的文件描述符可能会有副作用
(特别是涉及到错误处理、文件结尾条件等等的时候)。
在不同的操作系统上这种行为也是不一样，特别的，上面的例子都不能在非Unix系统上运行。
我说了这么多，意思就是让你充分测试自己的实现代码，确保它能按照期望工作。

## 5.19 创建临时文件和文件夹


### 问题


你需要在程序执行时创建一个临时文件或目录，并希望使用完之后可以自动销毁掉。

### 解决方案


tempfile 模块中有很多的函数可以完成这任务。
为了创建一个匿名的临时文件，可以使用 tempfile.TemporaryFile ：

In [ ]:
from tempfile import TemporaryFile

with TemporaryFile('w+t') as f:
    # Read/write to the file
    f.write('Hello World\n')
    f.write('Testing\n')

    # Seek back to beginning and read the data
    f.seek(0)
    data = f.read()

# Temporary file is destroyed

或者，如果你喜欢，你还可以像这样使用临时文件：

In [ ]:
f = TemporaryFile('w+t')
# Use the temporary file
...
f.close()
# File is destroyed

TemporaryFile() 的第一个参数是文件模式，通常来讲文本模式使用 w+t ，二进制模式使用 w+b 。
这个模式同时支持读和写操作，在这里是很有用的，因为当你关闭文件去改变模式的时候，文件实际上已经不存在了。
TemporaryFile() 另外还支持跟内置的 open() 函数一样的参数。比如：

In [ ]:
with TemporaryFile('w+t', encoding='utf-8', errors='ignore') as f:
    ...

在大多数Unix系统上，通过 TemporaryFile() 创建的文件都是匿名的，甚至连目录都没有。
如果你想打破这个限制，可以使用 NamedTemporaryFile() 来代替。比如：

In [ ]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile('w+t') as f:
    print('filename is:', f.name)
    ...

# File automatically destroyed

这里，被打开文件的 f.name 属性包含了该临时文件的文件名。
当你需要将文件名传递给其他代码来打开这个文件的时候，这个就很有用了。
和 TemporaryFile() 一样，结果文件关闭时会被自动删除掉。
如果你不想这么做，可以传递一个关键字参数 delete=False 即可。比如：

In [ ]:
with NamedTemporaryFile('w+t', delete=False) as f:
    print('filename is:', f.name)
    ...

为了创建一个临时目录，可以使用 tempfile.TemporaryDirectory() 。比如：

In [ ]:
from tempfile import TemporaryDirectory

with TemporaryDirectory() as dirname:
    print('dirname is:', dirname)
    # Use the directory
    ...
# Directory and all contents destroyed

### 讨论


TemporaryFile() 、NamedTemporaryFile() 和 TemporaryDirectory() 函数
应该是处理临时文件目录的最简单的方式了，因为它们会自动处理所有的创建和清理步骤。
在一个更低的级别，你可以使用 mkstemp() 和 mkdtemp() 来创建临时文件和目录。比如：

In [ ]:
import tempfile
tempfile.mkstemp()

In [ ]:
tempfile.mkdtemp()

但是，这些函数并不会做进一步的管理了。
例如，函数 mkstemp() 仅仅就返回一个原始的OS文件描述符，你需要自己将它转换为一个真正的文件对象。
同样你还需要自己清理这些文件。

通常来讲，临时文件在系统默认的位置被创建，比如 /var/tmp 或类似的地方。
为了获取真实的位置，可以使用 tempfile.gettempdir() 函数。比如：

In [ ]:
tempfile.gettempdir()

所有和临时文件相关的函数都允许你通过使用关键字参数
prefix 、suffix 和 dir 来自定义目录以及命名规则。比如：

In [ ]:
f = NamedTemporaryFile(prefix='mytemp', suffix='.txt', dir='/tmp')
f.name

最后还有一点，尽可能以最安全的方式使用 tempfile 模块来创建临时文件。
包括仅给当前用户授权访问以及在文件创建过程中采取措施避免竞态条件。
要注意的是不同的平台可能会不一样。因此你最好阅读
官方文档 来了解更多的细节。

## 5.20 与串行端口的数据通信


### 问题


你想通过串行端口读写数据，典型场景就是和一些硬件设备打交道(比如一个机器人或传感器)。

### 解决方案


尽管你可以通过使用Python内置的I/O模块来完成这个任务，但对于串行通信最好的选择是使用
pySerial包 。
这个包的使用非常简单，先安装pySerial，使用类似下面这样的代码就能很容易的打开一个串行端口：

In [ ]:
import serial
ser = serial.Serial('/dev/tty.usbmodem641', # Device name varies
                    baudrate=9600,
                    bytesize=8,
                    parity='N',
                    stopbits=1)

设备名对于不同的设备和操作系统是不一样的。
比如，在Windows系统上，你可以使用0, 1等表示的一个设备来打开通信端口”COM0”和”COM1”。
一旦端口打开，那就可以使用 read()，readline() 和 write() 函数读写数据了。例如：

In [ ]:
ser.write(b'G1 X50 Y50\r\n')
resp = ser.readline()

大多数情况下，简单的串口通信从此变得十分简单。

### 讨论


尽管表面上看起来很简单，其实串口通信有时候也是挺麻烦的。
推荐你使用第三方包如 pySerial 的一个原因是它提供了对高级特性的支持
(比如超时，控制流，缓冲区刷新，握手协议等等)。举个例子，如果你想启用 RTS-CTS 握手协议，
你只需要给 Serial() 传递一个 rtscts=True 的参数即可。
其官方文档非常完善，因此我在这里极力推荐这个包。

时刻记住所有涉及到串口的I/O都是二进制模式的。因此，确保你的代码使用的是字节而不是文本
(或有时候执行文本的编码/解码操作)。
另外当你需要创建二进制编码的指令或数据包的时候，struct 模块也是非常有用的。

## 5.21 序列化Python对象


### 问题


你需要将一个Python对象序列化为一个字节流，以便将它保存到一个文件、存储到数据库或者通过网络传输它。

### 解决方案


对于序列化最普遍的做法就是使用 pickle 模块。为了将一个对象保存到一个文件中，可以这样做：

In [ ]:
import pickle

data = ... # Some Python object
f = open('somefile', 'wb')
pickle.dump(data, f)

为了将一个对象转储为一个字符串，可以使用 pickle.dumps() ：

In [ ]:
s = pickle.dumps(data)

为了从字节流中恢复一个对象，使用 pickle.load() 或 pickle.loads() 函数。比如：

In [ ]:
# Restore from a file
f = open('somefile', 'rb')
data = pickle.load(f)

# Restore from a string
data = pickle.loads(s)

### 讨论


对于大多数应用程序来讲，dump() 和 load() 函数的使用就是你有效使用 pickle 模块所需的全部了。
它可适用于绝大部分Python数据类型和用户自定义类的对象实例。
如果你碰到某个库可以让你在数据库中保存/恢复Python对象或者是通过网络传输对象的话，
那么很有可能这个库的底层就使用了 pickle 模块。

pickle 是一种Python特有的自描述的数据编码。
通过自描述，被序列化后的数据包含每个对象开始和结束以及它的类型信息。
因此，你无需担心对象记录的定义，它总是能工作。
举个例子，如果要处理多个对象，你可以这样做：

In [ ]:
import pickle
f = open('somedata', 'wb')
pickle.dump([1, 2, 3, 4], f)
pickle.dump('hello', f)
pickle.dump({'Apple', 'Pear', 'Banana'}, f)
f.close()
f = open('somedata', 'rb')
pickle.load(f)

In [ ]:
pickle.load(f)

In [ ]:
pickle.load(f)

你还能序列化函数，类，还有接口，但是结果数据仅仅将它们的名称编码成对应的代码对象。例如：

In [ ]:
import math
import pickle.
pickle.dumps(math.cos)

当数据反序列化回来的时候，会先假定所有的源数据时可用的。
模块、类和函数会自动按需导入进来。对于Python数据被不同机器上的解析器所共享的应用程序而言，
数据的保存可能会有问题，因为所有的机器都必须访问同一个源代码。

注

In [ ]:
千万不要对不信任的数据使用pickle.load()。
pickle在加载时有一个副作用就是它会自动加载相应模块并构造实例对象。
但是某个坏人如果知道pickle的工作原理，
他就可以创建一个恶意的数据导致Python执行随意指定的系统命令。
因此，一定要保证pickle只在相互之间可以认证对方的解析器的内部使用。

有些类型的对象是不能被序列化的。这些通常是那些依赖外部系统状态的对象，
比如打开的文件，网络连接，线程，进程，栈帧等等。
用户自定义类可以通过提供 __getstate__() 和 __setstate__() 方法来绕过这些限制。
如果定义了这两个方法，pickle.dump() 就会调用 __getstate__() 获取序列化的对象。
类似的，__setstate__() 在反序列化时被调用。为了演示这个工作原理，
下面是一个在内部定义了一个线程但仍然可以序列化和反序列化的类：

In [ ]:
# countdown.py
import time
import threading

class Countdown:
    def __init__(self, n):
        self.n = n
        self.thr = threading.Thread(target=self.run)
        self.thr.daemon = True
        self.thr.start()

    def run(self):
        while self.n > 0:
            print('T-minus', self.n)
            self.n -= 1
            time.sleep(5)

    def __getstate__(self):
        return self.n

    def __setstate__(self, n):
        self.__init__(n)

试着运行下面的序列化试验代码：

In [ ]:
import countdown
c = countdown.Countdown(30)
T-minus 30

In [ ]:
# After a few moments
f = open('cstate.p', 'wb')
import pickle
pickle.dump(c, f)
f.close()

然后退出Python解析器并重启后再试验下：

In [ ]:
f = open('cstate.p', 'rb')
pickle.load(f)

你可以看到线程又奇迹般的重生了，从你第一次序列化它的地方又恢复过来。

pickle 对于大型的数据结构比如使用 array 或 numpy
模块创建的二进制数组效率并不是一个高效的编码方式。
如果你需要移动大量的数组数据，你最好是先在一个文件中将其保存为数组数据块或使用更高级的标准编码方式如HDF5
(需要第三方库的支持)。

由于 pickle 是Python特有的并且附着在源码上，所有如果需要长期存储数据的时候不应该选用它。
例如，如果源码变动了，你所有的存储数据可能会被破坏并且变得不可读取。
坦白来讲，对于在数据库和存档文件中存储数据时，你最好使用更加标准的数据编码格式如XML，CSV或JSON。
这些编码格式更标准，可以被不同的语言支持，并且也能很好的适应源码变更。

最后一点要注意的是  pickle 有大量的配置选项和一些棘手的问题。
对于最常见的使用场景，你不需要去担心这个，但是如果你要在一个重要的程序中使用pickle去做序列化的话，
最好去查阅一下 官方文档 。